In [250]:
%load_ext autoreload
import numpy as np
from pvlib import singlediode, pvsystem
from pvlib.tests.test_singlediode import get_pvsyst_fs_495

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [251]:
%autoreload 2

In [252]:
pvsyst_fs_495 = get_pvsyst_fs_495()
# reference conditions
poa, temp_cell, expected, tol = (
    pvsyst_fs_495['irrad_ref'],
    pvsyst_fs_495['temp_ref'],
    {
        'pmp': (pvsyst_fs_495['I_mp_ref'] *
                pvsyst_fs_495['V_mp_ref']),
        'isc': pvsyst_fs_495['I_sc_ref'],
        'voc': pvsyst_fs_495['V_oc_ref']
    },
    (5e-4, 0.04)
)

In [288]:
# first evaluate PVSyst model with thin-film recombination loss current
# at reference conditions
x = pvsystem.calcparams_pvsyst(
    effective_irradiance=poa, temp_cell=temp_cell,
    alpha_sc=pvsyst_fs_495['alpha_sc'],
    gamma_ref=pvsyst_fs_495['gamma_ref'],
    mu_gamma=pvsyst_fs_495['mu_gamma'], I_L_ref=pvsyst_fs_495['I_L_ref'],
    I_o_ref=pvsyst_fs_495['I_o_ref'], R_sh_ref=pvsyst_fs_495['R_sh_ref'],
    R_sh_0=pvsyst_fs_495['R_sh_0'], R_sh_exp=pvsyst_fs_495['R_sh_exp'],
    R_s=pvsyst_fs_495['R_s'],
    cells_in_series=pvsyst_fs_495['cells_in_series'],
    EgRef=pvsyst_fs_495['EgRef']
)
il_pvsyst, io_pvsyst, rs_pvsyst, rsh_pvsyst, nnsvt_pvsyst = x
voc_est_pvsyst = singlediode.estimate_voc(photocurrent=il_pvsyst,
                              saturation_current=io_pvsyst,
                              nNsVth=nnsvt_pvsyst)
vd_pvsyst = np.linspace(0, voc_est_pvsyst, 1000)
bishop88_args = dict(
    diode_voltage=vd_pvsyst, photocurrent=il_pvsyst,
    saturation_current=io_pvsyst, resistance_series=rs_pvsyst,
    resistance_shunt=rsh_pvsyst, nNsVth=nnsvt_pvsyst,
    d2mutau=pvsyst_fs_495['d2mutau'],
    NsVbi=singlediode.VOLTAGE_BUILTIN*pvsyst_fs_495['cells_in_series']
)

In [254]:
def assert_allclose(res, res_no_if, res_where):
    for a, b, c in zip(res, res_no_if, res_where):
        assert np.allclose(a, b)
        assert np.allclose(a, c)

In [255]:
def benchmark(breakdown_factor, bishop88_args, gradients=False, is_array=False):
    if not is_array:
        print('bishop88')
        %timeit res = singlediode.bishop88(**bishop88_args, breakdown_factor=breakdown_factor, gradients=gradients)
    print('bishop88_no_if')
    %timeit res_no_if = singlediode.bishop88_no_if(**bishop88_args, breakdown_factor=breakdown_factor, gradients=gradients)
    print('bishop88_where')
    %timeit res_where = singlediode.bishop88_where(**bishop88_args, breakdown_factor=breakdown_factor, gradients=gradients)
    assert_allclose(res, res_no_if, res_where)

# breakdown_factor is scalar

In [ ]:
num_curves = 20

In [292]:
def get_bishop88_scalar_args(num_curves):
    """
    Duplicate the one bishop88_args curve into many curves.
    """
    diode_voltage = np.tile(bishop88_args['diode_voltage'][:, None], num_curves)
    args = {k: np.tile(v, num_curves) for k, v in bishop88_args.items()}
    args['diode_voltage'] = diode_voltage
    return args

### breakdown_factor is 0.

In [294]:
benchmark(0., get_bishop88_scalar_args(num_curves))

bishop88
548 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
bishop88_no_if
994 µs ± 35.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
bishop88_where
631 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### breakdown_factor is .1

In [296]:
benchmark(.1, get_bishop88_scalar_args(num_curves))

bishop88
995 µs ± 27.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
bishop88_no_if
976 µs ± 2.74 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
bishop88_where
987 µs ± 2.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Gradients

### breakdown_factor is 0.

In [297]:
benchmark(0., get_bishop88_scalar_args(num_curves), gradients=True)

bishop88
1.07 ms ± 4.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
bishop88_no_if
2.4 ms ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
bishop88_where
1.39 ms ± 42.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### breakdown_factor is .1

In [298]:
benchmark(.1, get_bishop88_scalar_args(num_curves), gradients=True)

bishop88
2.49 ms ± 41 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
bishop88_no_if
2.55 ms ± 81.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
bishop88_where
2.58 ms ± 68.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# breakdown_factor is array

In [268]:
num_curves = 20

In [269]:
def get_bishop88_array_args(breakdown_factor, num_curves):
    """
    Duplicate the one bishop88_args curve into many curves.
    One curve for each breakdown_factor value.
    """
    keys, values = zip(*bishop88_args.items())
    breakdown_factor = np.tile(breakdown_factor, num_curves // 2)[:, None]
    breakdown_factor, *values = np.broadcast_arrays(breakdown_factor, *values)
    args = {k: v for k, v in zip(keys, values)}
    return breakdown_factor, args

### breakdown_factor is all zeros

In [270]:
breakdown_factor, bishop88_array_args = get_bishop88_array_args(np.array([0., 0.]), num_curves)
benchmark(breakdown_factor, bishop88_array_args, is_array=True)

bishop88_no_if
1.07 ms ± 55.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
bishop88_where
696 µs ± 21.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### breakdown_factor is [0, .1]

In [271]:
breakdown_factor, bishop88_array_args = get_bishop88_array_args(np.array([0., .1]), num_curves)
benchmark(breakdown_factor, bishop88_array_args, is_array=True)

bishop88_no_if
1.06 ms ± 27.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
bishop88_where
916 µs ± 5.53 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### breakdown_factor is [.1, .2]

In [272]:
breakdown_factor, bishop88_array_args = get_bishop88_array_args(np.array([.1, .2]), num_curves)
benchmark(breakdown_factor, bishop88_array_args, is_array=True)

bishop88_no_if
1.06 ms ± 26.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
bishop88_where
1.14 ms ± 76.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Gradients

### breakdown_factor is all zeros

In [273]:
breakdown_factor, bishop88_array_args = get_bishop88_array_args(np.array([0., 0.]), num_curves)
benchmark(breakdown_factor, bishop88_array_args, is_array=True, gradients=True)

bishop88_no_if
2.61 ms ± 61 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
bishop88_where
1.43 ms ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### breakdown_factor is [0, .1]

In [274]:
breakdown_factor, bishop88_array_args = get_bishop88_array_args(np.array([0., .1]), num_curves)
benchmark(breakdown_factor, bishop88_array_args, is_array=True, gradients=True)

bishop88_no_if
2.46 ms ± 65.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
bishop88_where
2.13 ms ± 183 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### breakdown_factor is [.1, .2]

In [275]:
breakdown_factor, bishop88_array_args = get_bishop88_array_args(np.array([.1, .2]), num_curves)
benchmark(breakdown_factor, bishop88_array_args, is_array=True, gradients=True)

bishop88_no_if
2.51 ms ± 55.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
bishop88_where
2.78 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
